# Bitcoin and Ethereum derivation, balances

In [1]:
from IPython.display import display
from ipywidgets import widgets, Layout
import time

from web3 import Web3, middleware
from web3.gas_strategies.time_based import fast_gas_price_strategy

from crypto_eth import HDPrivateKey, HDKey
import pandas as pd
from binascii import hexlify

In [2]:
phrase = ''

text = widgets.Text(
    description='Phrase:',
    layout=Layout(width="70%")
)
display(text)

def handle(sender):
    global phrase
    phrase = text.value
    text.value =""
    
text.on_submit(handle)

Text(value='', description='Phrase:', layout=Layout(width='70%'))

## Ethereum

In [3]:
master_key = HDPrivateKey.master_key_from_mnemonic(phrase)

root_keys = HDKey.from_path(master_key, "m/44'/60'/0'")
acct_priv_key = root_keys[-1]
private_keys = []
addresses = []

ETHLEN = 10

# derivation example for Ethereum generate first 10 addresses
for i in range(ETHLEN):
    keys = HDKey.from_path(acct_priv_key, '{change}/{index}'.format(change=0, index=i))
    private_key = keys[-1]
    public_key = private_key.public_key
    private_keys.append(private_key)
    addresses.append(private_key.public_key.address())
    

df = pd.DataFrame(addresses, columns = ['Address']) 

df

,Address
0,0x88a007ec4f1819f24c0988fc9c26496b99b436d1
1,0xab33d517b6a63a0b1c099b8438d6641cf1a984cc
2,0xb76d3a5d568f339411a6f0697694cddb6396df58
3,0xcb64cd644d24b57a96287f61dffeb402b01e23c6
4,0x8396738b4cb2a2a7ca636161346a7cb22ee06e25
5,0xbd666c0ec86628475b1602aba0ebf45570a33d4d
6,0x46ff31e07d54a28c468c300325afcdac4c6b7f2e
7,0x44874a7d255df37f5b9c91d173bee6ab4441c881
8,0x9a2c78cad9e6c0dc46820c37704065bb22e5eab0
9,0x577c31aad54366e7f8cf931a804f409d20b6928c


In [24]:
# use infura endpoint
# endpoint_url = 'wss://ropsten.infura.io/ws/adc06f70568e46d88376a8a3a30e5497'
endpoint_url = 'https://ropsten.infura.io/v3/adc06f70568e46d88376a8a3a30e5497'

# Ropsten chainId is 3
chainId = 3

# setup connection
w3 = Web3(Web3.HTTPProvider(endpoint_url))
if not w3.isConnected():
    print('Can\'t connect to Web3 endpoint')

# select gas price strategy
w3.eth.setGasPriceStrategy(fast_gas_price_strategy)

In [25]:
addresses = [w3.toChecksumAddress(key.public_key.address()) for key in private_keys]
balances = [w3.fromWei(w3.eth.getBalance(address), 'ether') for address in addresses]

if float(balances[0]) <= 0:
    raise Exception('Wrong phrase!')
df = pd.DataFrame(zip(addresses,balances), columns = ['Address', 'Balance'])
df

Exception: Wrong phrase!

## Bitcoin

In [6]:
from pybip44 import HDPrivateKey as BtcHDPrivateKey
import requests

from bitcoin_gp import BitcoinGP, Fee

In [27]:
master_key = BtcHDPrivateKey.master_key_from_mnemonic(phrase, 'btc')

# to switch between test and main network, you need to change 'coin' in derivation path m/44'/{coin}'/0
# for Main coin is 0
# for Test coin is 1
root_key = BtcHDPrivateKey.from_path(master_key, "m/44'/1'/0'")

private_keys = []
addresses = []

for i in range(10):
    private_key = BtcHDPrivateKey.from_path(root_key, '{change}/{index}'.format(change=0, index=i))
    private_keys.append(private_key.to_hex())
    public_key = private_key.public_key
#     print(f'\tPublic Key: {public_key.to_hex()}')
#     print(f'\tPrivate Key: {private_key.to_hex()}')
    
    # and here you need to change main_net parameter
    addresses.append(public_key.address(main_net=False))

    
df = pd.DataFrame(addresses, columns = ['Address']) 

df

,Address
0,mjhDqt76W6piQ7tHZZLid674TmC4c49tVn
1,mkx1ajkkvcAMxVbRNzuzEsVeznwMYJJB85
2,mosHSEacF7UCRgFCmPnr7u1aEUqpPRWfV6
3,mjXCAC3NNeJU3kiJ12KQbBY4LzBo4zzdL2
4,mfoYaaHVFckrcKgXJ7KQg4MHUeRARPJVo9
5,mqyfBSvJFZ7B5LCegcQosXkRrcpwcJJ1kY
6,mqHYvMSWy46YagiFj2uvyBSwKGY4uUC7Yz
7,n3uYpmQsqEFCrcwPgsdjpyeSPjCQSNf9Ey
8,mgvriZH7gY2YnBrQ5zfFPF5qH1QdHxtknm
9,mgrwCrAzdEqDYPosBJFYoSypL2QHwToymM


In [28]:
decimals = 10 ** 8

# here test net node on https://testgate.geniepay.io/v1/
# and main net on https://gate.geniepay.io/v1/
# url = 'https://gate.geniepay.io/v1/'
url = 'https://testgate.geniepay.io/v1/'
btc_gp = BitcoinGP(url, main_net = False)

balances_confirmed = []

balances_unconfirmed = []


for address in addresses:
    balance = btc_gp.get_balance(address)
    balances_confirmed.append(balance['confirmed'] / decimals)
    balances_unconfirmed.append(balance['unconfirmed'] / decimals)
    
if float(balances[0]) <= 0:
    raise Exception('Wrong phrase!')   
df = pd.DataFrame(zip(addresses, balances_confirmed, balances_unconfirmed), columns = ['Address', 'Balance (confirmed)', 'Balance (unconfirmed)']) 
df

Exception: Wrong phrase!